In [11]:
# our nice list of imports is in here
#i hate this, but the notebook won't recognize our custom modules w/o it
import sys
sys.path.append("..")
import lib
from lib import utils

import os
import cv2
import urllib
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow.keras.models import load_model


from mtcnn.mtcnn import MTCNN

## Loading the Model

In [12]:
model = load_model('../models/kaggle_relu_softmax_1e41e5')

In [13]:


os.listdir('../models')

['mnm',
 'keras_all_features_sequential_model',
 '.DS_Store',
 'kaggle_relu_softmax_1e41e5',
 'keras_half_features_sequential_model',
 'keras_8_feature_incorrect_mask_sequential_model',
 '.ipynb_checkpoints',
 '.mask_recog_ver2.h5.icloud',
 'image_model',
 'keras_sequential_model',
 'keras_mask_or_no_mask_model']

## Using Our Model With Video Feed

+ We have two available facial detection algorithms: 
  1. Haar Cascade - faster detection, but only tuned for full, frontal views of faces.
  2. MTCNN - slower detection, but returns extra features and can detect faces from a variety of angles -- not only the front.


In [17]:
# use haar cascade to detect faces (frontal)
# HC_FRONTAL_FACE = os.path.join(HC_DATA_DIR, 
#                                'haarcascade_frontalface_default.xml')
# classifier = cv2.CascadeClassifier(HC_FRONTAL_FACE)


# use MTCNN to detect faces
classifier = MTCNN()


labels_dict={ 0:'No Mask', 1:'Mask' }
color_dict={ 0: (0,0,255), 1: (0,255,0) }

size = 4

# open webcam feed
# cap = cv2.VideoCapture(0)

# alternatively, open feed from a file
cap = cv2.VideoCapture('../memask-beard.mov')

# EVEN from a mobile device (UNTESTED)
# cap = cv.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if frame is not None:
        # frame = cv2.flip(frame, 1, 1) # flip so view looks more natural live (?)
        

        # detection is a little faster on smaller images -- so let's make it small
        small = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))

        # detect faces with haar cascades
        # faces = classifier.detectMultiScale(small)
        
        # detect faces with MTCNN
        faces = classifier.detect_faces(small)

        for f in faces:
            # reverse the scale-down of the frame for bounding box (haar)
#             (x, y, w, h) = [v * size for vstack in f] 
#             keypoints = None
            
            # reverse the scale-down of the frame for bounding box (MTCNN)
            (x, y, w, h) = [v * size for v in f['box']]
            # print(f['keypoints'])
            keypoints = f['keypoints']
                
        
            face = frame[y:y+h, x:x+w]
            resized = cv2.resize(face, (50,50))
            normalized = resized/255.0
            reshaped = np.reshape(normalized,(-1,50,50,1))
            reshaped = np.vstack([reshaped])
            result = model.predict(reshaped)
            print(result)

            label = np.argmax(result, axis=1)[0]
#             print("PREDICTION: {}".format(labels_dict[label]))
            cv2.rectangle(frame, (x,y), (x+w,y+h), color_dict[label], 2)
            cv2.rectangle(frame, (x,y-40), (x+w,y), color_dict[label], -1)
            cv2.putText(
                frame, 
                labels_dict[label], 
                (x, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (255,255,255),
                2
            )
            
            # extra fun when using MTCNN :)
            if keypoints is not None:
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['left_eye']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['right_eye']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['nose']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['mouth_left']])), 
                    2, (255,0,255), 2
                )
                cv2.circle(
                    frame, 
                    (tuple([k * size for k in keypoints['mouth_right']])), 
                    2, (255,0,255), 2)
            # end of extra fun with MTCNN

        cv2.imshow('LIVE DETECTION ACTIVE', frame)
        key = cv2.waitKey(10)

        if key == 27: # if Esc key pressed -- end feed
            break
        
#         clear_output(wait=True)
    else:
        break
        
# stop video
cap.release()

# close windows
cv2.destroyAllWindows()

[[2.6909966e-04 9.9973089e-01]
 [9.9999273e-01 7.2571152e-06]
 [2.4421411e-03 9.9755788e-01]]
[[4.4549835e-01 5.5450165e-01]
 [1.0000000e+00 6.9950085e-10]
 [5.7673180e-01 4.2326817e-01]]
[[1.0000000e+00 1.3490080e-08]
 [1.0000000e+00 3.9938324e-13]
 [2.0862722e-01 7.9137278e-01]]
[[4.8134057e-03 9.9518663e-01]
 [9.9999917e-01 8.5046486e-07]
 [1.6787127e-02 9.8321289e-01]]
[[4.5141816e-01 5.4858184e-01]
 [1.0000000e+00 1.5545090e-12]
 [9.3955433e-01 6.0445685e-02]]
[[9.9998951e-01 1.0480813e-05]
 [1.0000000e+00 1.0343813e-12]
 [9.4841862e-01 5.1581439e-02]]
[[1.0000000e+00 1.8243689e-08]
 [1.0000000e+00 1.5559951e-12]
 [6.5340802e-02 9.3465918e-01]]
[[9.9895942e-01 1.0405564e-03]
 [1.0000000e+00 3.2699165e-12]
 [7.7197365e-02 9.2280263e-01]]
[[1.0115265e-02 9.8988473e-01]
 [9.9999976e-01 2.2781296e-07]
 [1.2958795e-01 8.7041205e-01]]
[[7.4531548e-03 9.9254686e-01]
 [9.9999976e-01 1.8372690e-07]
 [2.6990432e-01 7.3009568e-01]]
[[3.9623475e-05 9.9996042e-01]
 [1.0000000e+00 2.6093319e-09

[[4.1109598e-03 9.9588901e-01]
 [1.0000000e+00 1.9315446e-10]
 [8.7512416e-01 1.2487586e-01]]
[[9.9999499e-01 4.9959181e-06]
 [1.0000000e+00 4.4452952e-11]
 [5.8513951e-01 4.1486046e-01]]
[[1.0796124e-03 9.9892044e-01]
 [9.9908221e-01 9.1781357e-04]
 [1.6800174e-04 9.9983191e-01]]
[[9.9789941e-01 2.1006193e-03]
 [1.0000000e+00 5.8902666e-13]
 [7.4868208e-01 2.5131789e-01]]
[[4.4772159e-03 9.9552274e-01]
 [5.7208210e-05 9.9994278e-01]
 [9.9996769e-01 3.2302120e-05]]
[[7.5651860e-01 2.4348141e-01]
 [1.4421620e-13 1.0000000e+00]
 [9.9999666e-01 3.3077220e-06]]
[[1.7835209e-02 9.8216486e-01]
 [8.5271982e-05 9.9991477e-01]
 [2.6153741e-02 9.7384620e-01]]
[[5.9807212e-03 9.9401927e-01]
 [5.4859283e-05 9.9994516e-01]
 [9.6048459e-02 9.0395159e-01]]
[[0.01627586 0.9837241 ]
 [0.0138758  0.9861243 ]
 [0.12750275 0.87249726]]
[[1.4291642e-03 9.9857080e-01]
 [1.0000000e+00 1.2360223e-08]
 [5.3659564e-01 4.6340433e-01]]
[[6.0613616e-03 9.9393868e-01]
 [9.9966013e-01 3.3981403e-04]
 [2.5101611e-01 

[[9.7847186e-13 1.0000000e+00]
 [9.9999785e-01 2.1499125e-06]
 [5.3201216e-01 4.6798781e-01]]
[[1.5316257e-04 9.9984682e-01]
 [9.9998939e-01 1.0588005e-05]
 [1.1478490e-03 9.9885213e-01]]
[[3.6749005e-04 9.9963248e-01]
 [1.0000000e+00 1.1013712e-11]
 [7.2606704e-03 9.9273938e-01]]
[[1.8829999e-02 9.8117000e-01]
 [1.0000000e+00 1.0042837e-11]
 [9.3052577e-04 9.9906951e-01]]
[[9.9997544e-01 2.4543593e-05]
 [1.0000000e+00 1.2226909e-14]
 [6.2601209e-02 9.3739879e-01]]
[[9.4370138e-09 1.0000000e+00]
 [9.9999857e-01 1.4091927e-06]
 [5.0074649e-01 4.9925351e-01]]
[[5.6425552e-04 9.9943572e-01]
 [9.9999976e-01 1.9303870e-07]
 [4.6425976e-02 9.5357406e-01]]
[[8.0314121e-04 9.9919683e-01]
 [9.9999714e-01 2.8770305e-06]
 [8.1883913e-01 1.8116088e-01]]
[[9.99998569e-01 1.46388140e-06]
 [1.00000000e+00 4.16837219e-13]
 [8.77381682e-01 1.22618414e-01]]
[[6.5134169e-05 9.9993491e-01]
 [9.9979848e-01 2.0151037e-04]
 [8.6416489e-01 1.3583520e-01]]
[[9.7273085e-03 9.9027270e-01]
 [9.9999380e-01 6.17211

[[5.5552429e-01 4.4447571e-01]
 [6.5141910e-07 9.9999940e-01]
 [1.6759177e-05 9.9998319e-01]]
[[4.9198043e-01 5.0801957e-01]
 [5.4797363e-07 9.9999940e-01]
 [1.2108634e-04 9.9987888e-01]]
[[6.1167186e-01 3.8832808e-01]
 [9.2478518e-07 9.9999905e-01]
 [1.8107871e-04 9.9981898e-01]]
[[6.7864192e-01 3.2135803e-01]
 [7.0848574e-07 9.9999928e-01]
 [3.3047961e-04 9.9966955e-01]]
[[5.9646410e-01 4.0353587e-01]
 [7.7900717e-07 9.9999917e-01]
 [8.6852693e-04 9.9913150e-01]]
[[8.7525982e-01 1.2474020e-01]
 [1.8914210e-03 9.9810863e-01]
 [5.9131535e-06 9.9999404e-01]]
[[9.9995387e-01 4.6080349e-05]
 [1.5131482e-06 9.9999845e-01]
 [1.8304766e-03 9.9816954e-01]]
[[9.9999988e-01 1.4104198e-07]
 [4.6543417e-05 9.9995351e-01]
 [5.6740021e-07 9.9999940e-01]]
[[1.0000000e+00 2.6297643e-11]
 [4.6538317e-04 9.9953461e-01]
 [9.9685103e-06 9.9998999e-01]]
[[1.4896481e-02 9.8510355e-01]
 [3.4257862e-06 9.9999654e-01]
 [5.9687853e-01 4.0312141e-01]]
[[5.3759646e-01 4.6240354e-01]
 [5.2367548e-05 9.9994767e-01

[[6.8104320e-04 9.9931896e-01]
 [1.1282641e-04 9.9988711e-01]
 [3.3400080e-04 9.9966598e-01]]
[[1.2655816e-03 9.9873441e-01]
 [4.0108335e-04 9.9959892e-01]
 [1.5402215e-02 9.8459780e-01]]
[[2.7715398e-03 9.9722844e-01]
 [1.5713062e-04 9.9984288e-01]
 [5.1717332e-04 9.9948275e-01]]
[[4.5238356e-03 9.9547619e-01]
 [5.9478736e-04 9.9940526e-01]
 [7.8049721e-04 9.9921954e-01]]
[[6.4820386e-03 9.9351799e-01]
 [7.8972946e-08 9.9999988e-01]
 [2.7679550e-04 9.9972314e-01]]
[[1.20805821e-03 9.98791993e-01]
 [1.64170174e-07 9.99999881e-01]
 [1.35025575e-05 9.99986529e-01]]
[[8.5842954e-03 9.9141568e-01]
 [2.9587043e-05 9.9997044e-01]
 [1.3676587e-06 9.9999869e-01]]
[[2.2180475e-02 9.7781950e-01]
 [6.0091722e-08 9.9999988e-01]
 [2.9528229e-07 9.9999976e-01]]
[[1.4843010e-02 9.8515701e-01]
 [5.2381329e-06 9.9999475e-01]
 [3.5726745e-07 9.9999964e-01]]
[[2.325482e-03 9.976745e-01]
 [4.891856e-06 9.999951e-01]
 [8.393808e-08 9.999999e-01]]
[[9.2667993e-03 9.9073315e-01]
 [4.2587280e-07 9.9999952e-01

In [6]:
cap.release()
cv2.destroyAllWindows()